# Previsão com Prophet

In [1]:
#conda install -c conda-forge fbprophet

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from fbprophet import Prophet
import matplotlib.pyplot as plt

In [ ]:
N = 60 
i = 1800 

In [ ]:
all_data = pd.read_csv("SeriesTemporais/Download/Dados/train_1.csv").T 
key = pd.read_csv("key_1.csv")

In [ ]:
train, test = all_data.iloc[0:-N,:], all_data.iloc[-N:,:]
test_cleaned = test.T.fillna(method='ffill').T
train_cleaned = train.T.iloc[:,1:].fillna(method='ffill').T

In [ ]:
data=train_cleaned.iloc[:,i].to_frame()
data.columns = ['visits']
data['mean'] = pd.Series(data.visits).rolling(window=50).mean()
std_mult = 1.5
data.ix[np.abs(data.visits-data.visits.mean())>=(std_mult*data.visits.std()),'visits'] = data.ix[np.abs(data.visits-data.visits.mean())>=(std_mult*data.visits.std()),'mean']
data.index = pd.to_datetime(data.index)

In [ ]:
data.head()

In [ ]:
X = pd.DataFrame(index=range(0,len(data)))
X['ds'] = data.index
X['y'] = data['visits'].values
X.head()

In [ ]:
m = Prophet(yearly_seasonality=True)
m.fit(X)
future = m.make_future_dataframe(periods=N)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
m.plot(forecast);